In [1]:
import pickle, pickle5
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
with open('../Data/evdriver/preprocessed/meta.pickle', 'rb') as f:
    meta = pickle.load(f)
with open('../Data/evdriver/preprocessed/drivers.pickle', 'rb') as f:
    train_dv = pickle.load(f)
with open('../Data/evdriver/preprocessed/testset_df.pickle', 'rb') as f:
    test_dv = pickle.load(f)

In [3]:
test_dv = test_dv.sort_values(by=['Driver']).reset_index(drop=True)

In [5]:
meta.columns

Index(['statnm', 'statid', 'chgerid', 'chgertype', 'addr', 'location', 'lat',
       'lng', 'busiid', 'bnm', 'businm', 'busicall', 'powertype', 'output',
       'method', 'zcode', 'zscode', 'kind', 'kinddetail', 'parkingfree',
       'note', 'trafficyn', 'speed', 'Count', 'Cluster'],
      dtype='object')

In [6]:
meta

,statnm,statid,chgerid,chgertype,addr,location,lat,lng,busiid,bnm,...,zcode,zscode,kind,kinddetail,parkingfree,note,trafficyn,speed,Count,Cluster
0,암사1동 공영주차장,0,01,6,서울특별시 강동구 고덕로28길 36,null,37.552909,127.137255,63,환경부,...,11,11740,B0,B001,N,None,N,1,1,349
1,천호유수지 공영주차장,1,01,6,서울특별시 강동구 선사로 70,null,37.544812,127.121840,63,환경부,...,11,11740,B0,B001,N,None,N,1,1,347
2,천호3동 제1공영주차장,2,01,6,서울특별시 강동구 진황도로29길 34,null,37.538795,127.133399,63,환경부,...,11,11740,B0,B001,N,None,N,1,1,347
3,가로공원로 지하공영주차장,3,01,6,서울특별시 강서구 가로공원로 189,null,37.537263,126.838379,63,환경부,...,11,11500,B0,B001,N,None,N,1,1,4
4,곰달래문화복지센터 공영주차장,4,01,6,서울특별시 강서구 강서로5길 50,null,37.528956,126.849089,63,환경부,...,11,11500,B0,B001,N,None,N,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32052,춘천우체국,30891,01,2,강원특별자치도 춘천시 방송길 108,null,37.867039,127.720537,21,이지차저,...,51,51110,A0,A003,N,관리 시설에 따른 이용 제한이 있을 수 있음,N,0,1,462
32053,시흥시청,30892,01,4,경기도 시흥시 시청로 20,지상1층 주차장,37.379704,126.801784,16,채비,...,41,41390,A0,A001,N,None,N,1,6,42
32054,군포동아아파트,30893,01,2,경기도 군포시 군포로490번길 22,지상1층 주차장,37.350012,126.946231,16,채비,...,41,41410,H0,H001,Y,None,N,0,7,66
32055,반월호수주차타워,30894,01,6,경기도 군포시 호수로 97,실내주차장,37.325129,126.890020,16,채비,...,41,41410,B0,B001,N,None,N,1,2,62


In [6]:
def load_and_concatenate_batches(save_dir, n_batches):
    batches = []
    for u_batch_id in range(n_batches):
        batch_file = os.path.join(save_dir, f'batch_{u_batch_id}.npy')
        rate_batch = np.load(batch_file)
        batches.append(rate_batch)
    entire_rank = np.concatenate(batches, axis=0)
    return entire_rank

# 전체 배치 개수 계산
n_batches = 25
save_dir = './rank_batches2'
# 저장된 배치 배열을 읽어서 결합
entire_rank = load_and_concatenate_batches(save_dir, n_batches)

In [7]:
len(entire_rank)

51200

In [4]:
def evaluation(result, driver_id, test_dv, meta, criteria):
    location = test_dv.loc[test_dv['Driver'] == driver_id][criteria].values[0]
    teststat = test_dv.loc[test_dv['Driver'] == driver_id]['statid'].values[0]
    speed = test_dv.loc[test_dv['Driver'] == driver_id]['speed'].values
    if len(speed) == 1:
        flag = True
    else:
        flag = False

    driver_pref = result[driver_id]
    #[1011, 14000, 13000] -> statid의 list
    driver_rank = driver_pref.argsort()[::-1].astype(str)
    #하나의 Cluster로 위치 필터링
    driver_meta = meta[meta[criteria] == location]
    recommended_df = pd.DataFrame([])

    for i in range(len(driver_rank)):
        if len(recommended_df) > 19:
            false_count = len(recommended_df) - recommended_df.duplicated('statid').sum()
            if false_count == 20:
                break
        #추천된 statid가 Cluster에 속하면,
        if driver_rank[i] in driver_meta['statid'].values:
            stat_driver = driver_meta.loc[driver_meta['statid'] == driver_rank[i]]
            #추천된 statid의 speed가 testset의 speed와 같으면,
            if flag:
                if stat_driver['speed'].values[0] == speed:
                    recommended_df = pd.concat([recommended_df, stat_driver])
            else:
                recommended_df = pd.concat([recommended_df, stat_driver])
                
    if flag:
        recommended_df = recommended_df.loc[recommended_df['speed'] == speed[0]].reset_index(drop=True)
    else:
        recommended_df = recommended_df.drop_duplicates('statid').reset_index(drop=True)

    try:
        rank = recommended_df.loc[recommended_df['statid'] == teststat].index[0] 
        rank = rank + 1
    except:
        rank = 0
    return recommended_df, rank, len(recommended_df)

In [5]:
def get_ndcg(recommended_df, test_dv, driver_id, K):
    busiid = test_dv.loc[test_dv['Driver'] == driver_id]['busiid'].values[0]
    ex = (recommended_df['busiid'] == busiid).values
    
    ndcgs = []
    for j in K:
        if len(ex) >= j:
            dcg = 0
            for i in range(j):
                if ex[i] == True:
                    dcg += 1/np.log2(2 + i)

            idcg = 0
            sum_ex = ex.sum()
            if sum_ex >= j:
                for i in range(j):
                    idcg += 1/np.log2(2 + i)
            else:
                for i in range(sum_ex):
                    idcg += 1/np.log2(2 + i)

            if idcg == 0:
                ndcgs.append(0)
            else:
                ndcgs.append(dcg/idcg)
        else:
            ndcgs.append(ndcgs[-1])

    return ndcgs

In [17]:
ndcgs = []
for i in tqdm(range(10000)):
    a, b, c = evaluation(entire_rank, i, test_dv, meta, 'zscode')
    try:
        ndcg = get_ndcg(a, test_dv, i, [1, 5, 10, 15, 20])
        ndcgs.append(ndcg)
    except:
        print(i)

100%|██████████| 10000/10000 [1:00:26<00:00,  2.76it/s]


In [18]:
ndcgs = np.array(ndcgs)
np.save('./rank_batches2/ndcgs_zscode_10000.npy', ndcgs)

In [20]:
zscodendcg = np.load('./rank_batches2/ndcgs_zscode_10000.npy')

In [21]:
zscodendcg.mean(axis=0)

array([0.1165    , 0.15634209, 0.23074116, 0.29733332, 0.35604914])

In [37]:
ndcgs1 = []
for i in tqdm(range(10000)):
    a, b, c = evaluation(entire_rank, i, test_dv, meta, 'Cluster')
    try:
        ndcg = get_ndcg(a, test_dv, i, [1, 5, 10, 15 ,20])
        ndcgs1.append(ndcg)
    except:
        print(i)

100%|██████████| 10000/10000 [2:02:24<00:00,  1.36it/s] 


In [38]:
ndcgs1 = np.array(ndcgs1)
np.save('./rank_batches2/ndcgs_cluster_10000.npy', ndcgs1)

In [41]:
clusterndcg = np.load('./rank_batches2/ndcgs_cluster_10000.npy')

In [42]:
clusterndcg.mean(axis=0)

array([0.1815    , 0.25604018, 0.34383757, 0.40077141, 0.44598624])

In [22]:
def precision(rank, K):
    pres_K = []
    for k in K:
        pres = []
        for j in rank:
            if (j[0] <= k) & (j[0] != 0):
                pres.append(1)
            else:
                pres.append(0)
        pres_K.append(np.sum(pres))
    return pres_K

In [24]:
pres = precision(zscodendcg, [1, 5, 10, 15, 20])

In [25]:
pres

[1165, 1165, 1165, 1165, 1165]

In [26]:
zscodendcg

array([[0.        , 0.1695801 , 0.1695801 , 0.25866037, 0.49831084],
       [0.        , 0.        , 0.        , 0.26264954, 0.26264954],
       [0.        , 0.        , 0.27691895, 0.40017479, 0.40017479],
       ...,
       [0.        , 0.        , 0.        , 0.        , 0.24465054],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.4776237 , 0.63404995, 0.63404995, 0.63404995]])

In [28]:
a, b, c = evaluation(entire_rank, 0, test_dv, meta, 'zscode')

In [51]:
a['addr']

0        경기도 용인시 수지구 포은대로59번길 18
1           경기도 용인시 수지구 성복2로 220
2         경기도 용인시 처인구 남사읍 한숲로 83
3         경기도 용인시 수지구 만현로67번길 20
4            경기도 용인시 수지구 성복2로 10
5            경기도 용인시 수지구 성복1로 91
6         경기도 용인시 기흥구 구성로 105-15
7            경기도 용인시 기흥구 삼성2로 93
8           경기도 용인시 기흥구 강남동로 128
9           경기도 용인시 수지구 신봉1로 110
10           경기도 용인시 수지구 죽전로 267
11    경기도 용인시 기흥구 동백죽전대로527번길 80
12           경기도 용인시 기흥구 구성로 357
13        경기도 용인시 수지구 만현로67번길 20
14            경기도 용인시 기흥구 공세로 76
15      경기도 용인시 기흥구 중부대로746번길 21
16          경기도 용인시 기흥구 동백중앙로 41
17      경기도 용인시 기흥구 동백8로131번길 20
18            경기도 용인시 기흥구 예현로 15
19        경기도 용인시 기흥구 사은로84번길 10
Name: addr, dtype: object

In [30]:
b

0

In [31]:
c

20

In [68]:
test_dv.loc[test_dv['Driver'] == 0]['statid'].values[0]

'840'

In [66]:
a, b, c = evaluation(entire_rank, 0, test_dv, meta, 'Cluster')

In [69]:
a['statid']

0      839
1    20794
2    19776
3      815
4      840
Name: statid, dtype: object

In [63]:
for k in test_dv['statid'].unique():
    if k in a['statid'].unique():
        print(k)
    else:
        pass

19711
19712
1373
19706
1935
19707
20784
20795
496
569
20796
571
772
20789


In [8]:
def get_recall(recommended_df, test_dv, driver_id, K):
    criteria = test_dv.loc[test_dv['Driver'] == driver_id]['statid'].values[0]
    ex = recommended_df['statid']
    
    pres_K = []
    for k in K:
        if (criteria in list(ex[:k])):
            pres_K.append(1)
        else:
            pres_K.append(0)
    return np.array(pres_K)

In [9]:
recalls = np.zeros(5)
for i in tqdm(range(51200)):
    a, b, c = evaluation(entire_rank, i, test_dv, meta, 'zscode')
    try:
        recall = get_recall(a, test_dv, i, [1, 5, 10, 15, 20])
        recalls += recall
    except:
        print(i)

100%|██████████| 51200/51200 [5:12:16<00:00,  2.73it/s]   


In [112]:
recalls/10000

array([0.024 , 0.0805, 0.1411, 0.193 , 0.239 ])

In [10]:
recalls/51200

array([0.02417969, 0.08464844, 0.14335938, 0.19392578, 0.24007813])

In [47]:
recalls

array([0., 0., 0., 0., 0.])

In [113]:
recalls1 = np.zeros(5)
for i in tqdm(range(51200)):
    a, b, c = evaluation(entire_rank, i, test_dv, meta, 'Cluster')
    try:
        recall = get_recall(a, test_dv, i, [1, 5, 10, 15, 20])
        recalls1 += recall
    except:
        print(i)

100%|██████████| 10000/10000 [2:02:36<00:00,  1.36it/s] 


In [114]:
recalls1/10000

array([0.0744, 0.2803, 0.4456, 0.5438, 0.6118])

In [ ]:
recalls1/51200

In [92]:
len(entire_rank)

10240

In [106]:
a, b, c = evaluation(entire_rank, 0, test_dv, meta, 'Cluster')

In [107]:
recall = get_recall(a, test_dv, 0, [1, 5, 10, 15, 20])

In [108]:
recall

array([0, 1, 1, 1, 1])

In [109]:
recalls1 += recall

In [110]:
recalls1

array([0., 1., 1., 1., 2.])